In [1]:
!pip install opendatasets --upgrade
import opendatasets as od
dataset_url = 'https://www.kaggle.com/datasets/gpreda/chinese-mnist'
od.download(dataset_url)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: rushiky
Your Kaggle Key: ··········


100%|██████████| 16.5M/16.5M [00:02<00:00, 8.01MB/s]


In [17]:
pip install torchsummary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np
import pandas as pd 
import torch
from torch import optim
import torch.nn as nn

import cv2
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score as a_s
import os 
from torchsummary import summary
import zipfile

In [5]:
df = pd.read_csv('chinese-mnist/chinese_mnist.csv',sep=',')

In [18]:

df.isnull().sum(),df.shape 
df.info()
X = df.drop('character', axis=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   suite_id   15000 non-null  int64 
 1   sample_id  15000 non-null  int64 
 2   code       15000 non-null  int64 
 3   value      15000 non-null  int64 
 4   character  15000 non-null  object
dtypes: int64(4), object(1)
memory usage: 586.1+ KB


In [19]:
class CustomDataset(Dataset):
  def __init__(self, k,l, csv_file ='chinese-mnist/chinese_mnist.csv'):


    self.df = pd.read_csv(csv_file)
    

    self.k= {'九':int(9),'十':int(10),'百':int(11),'千':int(12),'万':int(13),'亿':int(14),'零':int(0),'一':int(1),'二':int(2),'三':int(3),'四':int(4),'五':int(5),'六':int(6),'七':int(7),'八':int(8)}

    


    self.target = 'character'
    self.features = ['suite_id','sample_id','code',]


    self.labels   = np.asarray(self.df.iloc[:, 4])
    
    self.y = df[self.target]
    self.X = df.drop(self.target, axis =1 )

  def __getitem__(self,idx):

    single_image_label = self.labels[idx]

    class_id = self.k[single_image_label]
    
    img = Image.open(f"chinese-mnist/data/data/input_{self.X.iloc[idx, 0]}_{self.X.iloc[idx, 1]}_{self.X.iloc[idx, 2]}.jpg")
    img = np.array(img) # convert to np.array
       

    return img, class_id

  def __len__(self):

    return len(self.X)


In [20]:
train_df = df.groupby('value').apply(lambda x: x.sample(700, random_state=42)).reset_index(drop=True)
x_train, y_train  = train_df.iloc[:, :-2], train_df.iloc[:, -2]

test_df  = df.groupby('value').apply(lambda x: x.sample(300, random_state=42)).reset_index(drop=True)
x_test, y_test    = test_df.iloc[:, :-2], test_df.iloc[:, -2]
 
train_ds = CustomDataset(x_train, y_train)
trainloader = DataLoader(dataset=train_ds,batch_size=32, shuffle=True)

test_ds = CustomDataset(x_test, y_test)
testloader = DataLoader(test_ds, batch_size=32, shuffle=True)

In [21]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        
        self.fc1 = nn.Linear(64*13*13, 500)
        self.fc2 = nn.Linear(500, 15)

        self.log_softmax = nn.LogSoftmax(dim=0)
        self.conv2_drop = nn.Dropout2d()

    def forward(self, x):
        x = x.unsqueeze(1)
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        output = self.log_softmax(x)
        return output

model = Net()

In [22]:


los = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001,)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

In [ ]:
epochs = 5
print_every = 96

for e in range(epochs):
    running_loss = 0
    print(f"Epoch: {e+1}/{epochs}")

    for i, (images, labels) in enumerate(iter(trainloader)):

        
        images = images.resize_(images.size()[0], 64,64).float()
        optimizer.zero_grad()
        
        output = model.forward(images)   
        losss = los(output, labels)     
        losss.backward()                 
        optimizer.step()                 
        
        running_loss += losss.item()
        
        if i % print_every == 0:
            print(f"\tIteration: {i}\t Loss: {running_loss/len(trainloader):.3f}")
            running_loss = 0

Epoch: 1/5
	Iteration: 0	 Loss: 0.008
	Iteration: 96	 Loss: 0.441
	Iteration: 192	 Loss: 0.267
	Iteration: 288	 Loss: 0.203


In [13]:
test_size = len(test_ds)
train_size = len(train_ds)
r_labels = np.array([])
preds  = np.array([])

correct_preds = np.array([])


for i, (images, labels) in enumerate(iter(testloader)):
  model.eval()

    
  images = images.resize_(images.size()[0], 64,64).float()

  outputs = model(images)

  loss = los(outputs, labels)



  r_labels = np.concatenate((r_labels, labels.numpy()))


  for index, item in enumerate(outputs):
      if r_labels[index] == torch.argmax(item):
          correct_preds += 1

      preds  = np.concatenate((preds, torch.argmax(item).unsqueeze(-1).detach().numpy()))


conf_mat = confusion_matrix(r_labels, preds)


accuracy_scores= a_s(r_labels, preds)

print(f'Test results with respect to confusion matrix: \n{conf_mat} \nand the accuracy score is : \n {accuracy_scores}')

Test results with respect to confusion matrix: 
[[977   0   0   2   2   4   2   1   3   0   0   3   3   2   1]
 [  0 991   3   0   0   0   1   0   0   1   0   2   0   2   0]
 [  1  13 968  15   1   0   1   1   0   0   0   0   0   0   0]
 [  1   1  95 886   0  12   1   0   1   0   0   0   1   1   1]
 [  7   2   1   0 961   0   1   1  11   1   3  11   0   0   1]
 [  3   0   2  33   1 943   1   0   1   3   0   7   2   0   4]
 [ 10   4   8   6   3   4 927   1  15   4   5   3   4   5   1]
 [  3   1   0   4   2   2   0 963   3   9   2   3   2   0   6]
 [  0   1   0   0   0   0   3   0 994   1   0   1   0   0   0]
 [  2   0   2   1   8   8  10  11  13 915   3   2   2   2  21]
 [  0   4   0   0   2   4   2   2  10   4 956   0  15   1   0]
 [  5   2   4   1  19  12   0   2   4   4   0 930   3  10   4]
 [  8   1   5   5   1   2  10   1   7   0  27   6 923   3   1]
 [  4   1   1   1   3   4   2   0   2   1   1  10   1 968   1]
 [  3   1   2   2   8   9   2   2  15  20   1   4   1   2 928]] 
and t